# Monitoring a Keras model with TensorBoard (TB)

### What is TensorBoard?

TensorBoard is a web-based tool that lets us visualize our model's structureand monitor its training.

#### How do we use TensorBoard?

To use TensorBoard we need our Keras model to write log files in the format that TensorBoard can read. TB uses the information in these log files to generate visualizations, let's add TB logging to our Keras model. We already have our model built out that is named 'trained_model.h5'. Using the same code, add in the new section.

1. Create a TensorFlow logger object between defining the model and training the model

### Create a TensorFlow object in the model built earlier

In [ ]:
import pandas as pd 
import keras
from keras.models import Sequential
from keras.layers import *

# Get data
train = pd.read_csv('Data/sales_data_training_scaled.csv')
test = pd.read_csv('Data/sales_data_test_scaled.csv')

# Split train data into X and y input arrays
X = train.drop('total earnings', axis=1).values
y = train[['total earnings']].values

# Create a new sequential nn (keras sequential API)
model = Sequential()

# Add in dense layers and input nodes
model.add(Dense(50, input_dim=9, activation='relu', name='layer_1'))
model.add(Dense(100, activation='relu', name='layer_2')) # second layer
model.add(Dense(50, activation='relu', name='layer_3')) # third layer
model.add(Dense(1, activation='linear', name='output_layer')) # output layer, predicts 1 value

# Compile the model
model.compile(loss='mean_squared_error', optimize='adam')

# Create a TensorBoard logger object
logger = keras.callbacks.TensorBoard(
    log_dir='logs',    # pass it a log file parameter; logs = folder name
    write_graph=True,  # log the structure of the model
    histogram_freq=5   # statistics on layers; passes through the training data it will log statistics
)

# Train the model (tell model to use logger)
model.fit(X,                 # training features
          y,                 # expected output
          epochs=50,         # training passes (epoch)
          shuffle=True,      # works best when randomly shuffled
          verbose=2,         # print more detailed info
          callbacks=[logger] # a list of functions we want Keras to call in an array; can have multiple functions
)

# Split test data into X and y input arrays
X_test = test.drop('total_earnings', axis=1).values
y_test = test[['total_earnings']].values

# Measure the error rate of the testing data
test_error_rate = model.evaluate(X_test, y_test, verbose=0) # pass in verbose 0

# Print error rate
print('The mean squared error (MSE) for the test data set is: {}'.format(test_error_rate))

### TensorBoard Web Page

1. After running the model_logging code, we can open up a terminal on where the file is located
2. Run the following code: 
    
    - tensorboard --logdir=02-deep-learning-course/logs # this is the log folder location
    

3. Copy and paste the address into a web page to view TensorBoard's computational graph
4. Examine the graphs
    - You can see that the main input is connected to layer_1, layer_2, layer_3, and finally the output. If the layers have the same color then that means they have the same internal structure
    - The output layer is different because it uses a different activation function
    - Zooming into the small line you can see numbers which represents a __tensor__ or an array of data being passed between the layers
        - The numbers here represent the size of the tensor or array
        - The initial inputs for this neural network are nine values that get passed to the first layer
        - The question mark is the __batch size__ which means that TensorFlow can process batches of data at once
        - It's a question mark because you can change the batch size depending on how much data ou want to process at once
    - We can also expand the node by clicking layer_1 and expand it
        - Inside the node are actions being done insight of this specific layer of the neural network
5. Tracing the path of data through the graph
    - Click the output_layer and click on 'Trace inputs' on the left side
        - This highlights the path data flows through to generate a prediction from the neural network

### Visualize Training Progress

In [ ]:
import pandas as pd 
import keras
from keras.models import Sequential
from keras.layers import *

# Set run name for log folder
RUN_NAME = 'run_1_with_50_nodes'
RUN_NAME_2 = 'run_2_with_5_nodes'

# Get data
train = pd.read_csv('Data/sales_data_training_scaled.csv')
test = pd.read_csv('Data/sales_data_test_scaled.csv')

# Split train data into X and y input arrays
X = train.drop('total earnings', axis=1).values
y = train[['total earnings']].values

# Create a new sequential nn (keras sequential API)
model = Sequential()

# Add in dense layers and input nodes
model.add(Dense(50, input_dim=9, activation='relu', name='layer_1'))
model.add(Dense(100, activation='relu', name='layer_2')) # second layer
model.add(Dense(50, activation='relu', name='layer_3')) # third layer
model.add(Dense(1, activation='linear', name='output_layer')) # output layer, predicts 1 value

# Compile the model
model.compile(loss='mean_squared_error', optimize='adam')

# Create a TensorBoard logger object
logger = keras.callbacks.TensorBoard(
    log_dir='logs/{}'.format(RUN_NAME),  # pass it a log file parameter; logs = folder name
    write_graph=True,                    # log the structure of the model
    histogram_freq=5                     # statistics on layers; passes through the training data it will log statistics
)

# Train the model (tell model to use logger)
model.fit(X,                 # training features
          y,                 # expected output
          epochs=50,         # training passes (epoch)
          shuffle=True,      # works best when randomly shuffled
          verbose=2,         # print more detailed info
          callbacks=[logger] # a list of functions we want Keras to call in an array; can have multiple functions
)

# Split test data into X and y input arrays
X_test = test.drop('total_earnings', axis=1).values
y_test = test[['total_earnings']].values

# Measure the error rate of the testing data
test_error_rate = model.evaluate(X_test, y_test, verbose=0) # pass in verbose 0

# Print error rate
print('The mean squared error (MSE) for the test data set is: {}'.format(test_error_rate))

__________________________________

## Using a Trained Keras Model in Google Cloud

Now, you want to be able to scale it up in production to serve lots of users. What should you do? Since we're using Keras with TensorFlow backend, we can export our Keras model as a TensorFlow model and once we have a TensorFlow model we can upload that to Google Cloud ML service. Using the Google Cloud ML service, we can support as many users as we need.

### Exporting TensorFlow model

Add this code after building the model from above and testing it.

In [ ]:
# Create a SavedModelBuilder object
model_builder = tf.saved_model.builder.SavedModelBuilder('exported_model') # pass in a folder name to save in

# Declare model inputs and outputs 
# Keras makes it easy as it keeps track of i/o of the model so we just need to pass out the TensorFlow

# Pass in model.input as the name of the input for Tf to use
inputs = {
    'input': tf.saved_model.utils.build_tensor_info(model.input) # name of the input
}

# Pass in the model_output
outputs = {
    'earnings': tf.saved_models.utils.build_tensor_info(model.output)
}

# Create a Tf signature def: a function declaration in the programming language
# Tf looks for this to know how to run the prediction function of our model
signature_def = tf.saved_model.signature_def_utils.build_signature_def(
    inputs=inputs,
    outputs=outputs,
    method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
)

# Save both the structure of our model and current weights
# Pass in the reference to the current Keras session and assign the model a special
#    tag to make sure Tf knows this model is meant for serving users
# Pass in the signature def we just created
model_builder.add_meta_graph_and_varibles(
    K.get_session(),
    tags=[tf.saved_model.tag_constants.SERVING],
    signature_def_map={
        tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature_def
    }
)

# Save (see newly created folder)
model_builder.save()

### Notes
I created the new folder called exported_model. 

1. First, we have the saved_model, that pb file. This file contains the structure of our model in Google's protobuff format. 

2. There's also a variable sub-folder. This contains a checkpoint of the train weights from our neural network. 

Alright, this model's now ready to uploaded to the Google Cloud.